# Proof of Concept for all-active models as starting point for selective optimizations

In [9]:
%matplotlib inline
from ateamopt.utils import utility
from ateamopt.analysis.optim_analysis import Optim_Analyzer
from ateamopt.model_parameters import AllActive_Model_Parameters as ParamsHandler
import pprint
import os
import logging
from man_opt.utils import getModel
from allensdk.core.cell_types_cache import CellTypesCache


logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

pp = pprint.PrettyPrinter(indent=4)
rev_potential = utility.rev_potential
section_map = utility.bpopt_section_map

In [10]:
cell_id = 468193142
ctc = CellTypesCache()
model_param_file = f'{cell_id}_fit.json'
getModel(cell_id, download_path = model_param_file)
morph_file = f'{cell_id}.swc'
ctc.get_reconstruction(cell_id, morph_file)

param_handler = ParamsHandler(cell_id, swc_path = morph_file, released_aa_model_path = model_param_file)
model_params_bpopt = param_handler.get_release_params(section_map, rev_potential)
repeat_params = ['Ra', 'g_pas', 'e_pas'] # parameters that are uniform across the sections
idx_remove = []
seen_params = []
for i, param_dict in enumerate(model_params_bpopt):
    if param_dict['param_name'] in repeat_params and param_dict['param_name'] not in seen_params:
        param_dict['sectionlist'] = 'all'
        seen_params.append(param_dict['param_name'])
    elif param_dict['param_name'] in seen_params:
        idx_remove.append(i)
        
model_params_bpopt = [param_dict for j, param_dict in enumerate(model_params_bpopt) if j not in idx_remove]
pp.pprint(model_params_bpopt)

[   {   'dist_type': 'uniform',
        'mech': 'Im',
        'param_name': 'gbar_Im',
        'sectionlist': 'basal',
        'type': 'range',
        'value': 0.00687447546862},
    {   'dist_type': 'uniform',
        'mech': 'Ih',
        'param_name': 'gbar_Ih',
        'sectionlist': 'somatic',
        'type': 'range',
        'value': 1.66485101624e-05},
    {   'dist_type': 'uniform',
        'param_name': 'e_pas',
        'sectionlist': 'all',
        'type': 'section',
        'value': -83.467040608},
    {   'dist_type': 'uniform',
        'param_name': 'cm',
        'sectionlist': 'axonal',
        'type': 'section',
        'value': 5.11878173185},
    {   'dist_type': 'uniform',
        'mech': 'CaDynamics',
        'param_name': 'decay_CaDynamics',
        'sectionlist': 'axonal',
        'type': 'range',
        'value': 914.33998306},
    {   'dist_type': 'uniform',
        'mech': 'Ca_LVA',
        'param_name': 'gbar_Ca_LVA',
        'sectionlist': 'somatic',
        

## Add dendritic constraints

Unpublished data from Adam Shai [Shai et al, 2015]

![](bAP.png)

We will use dend vs soma amplitude at 100 and 300 microns away.
@100 microns and 300 microns dend/soma ~ 0.8 and 0.5 respectively.
Let's add these recordings for the maximal amplitude recording - LongDC 47

In [11]:
protocols_path = 'config/protocols.json'
features_path = 'config/features.json'

protocols = utility.load_json(protocols_path)
features = utility.load_json(features_path)
dend_locations = [100, 300]
select_proto_name = 'LongDC_47'
select_feature_name = 'AP_amplitude_from_voltagebase'
reduced_feature_set = [select_feature_name, 'voltage_base', 'AP_width', 'Spikecount']

dend_to_soma_ratio = {100 : 0.8, 300 : 0.5}

for proto_name, proto_def in protocols.items():
    if proto_name == select_proto_name:
        proto_def['extra_recordings'] = []
        for i, dend_location in enumerate(dend_locations):
            proto_def['extra_recordings'].append(
            {
                'var' : 'v',
                'somadistance' : dend_location,
                'type' : 'somadistance',
                'name' : 'dend%s'%i,
                'seclist_name' : 'apical'
            }
            )
            
            select_feature_mean, select_feature_std = features[proto_name]['soma'][select_feature_name]
            features[proto_name]['dend%s'%i] = {
                select_feature_name : [dend_to_soma_ratio[dend_location]*select_feature_mean,
                                      select_feature_std]
                
            }

protocols_new = {proto_name : proto_def for proto_name, proto_def in protocols.items() if proto_name == select_proto_name}
features_new = {proto_name : feature_def for proto_name, feature_def in features.items() if proto_name == select_proto_name}
features_new[select_proto_name]['soma'] = {feat_name : feat_stat for feat_name, feat_stat 
                            in features_new[select_proto_name]['soma'].items() if feat_name in reduced_feature_set}         
pp.pprint(protocols_new)
pp.pprint(features_new)

{   'LongDC_47': {   'extra_recordings': [   {   'name': 'dend0',
                                                 'seclist_name': 'apical',
                                                 'somadistance': 100,
                                                 'type': 'somadistance',
                                                 'var': 'v'},
                                             {   'name': 'dend1',
                                                 'seclist_name': 'apical',
                                                 'somadistance': 300,
                                                 'type': 'somadistance',
                                                 'var': 'v'}],
                     'stimuli': [   {   'amp': 0.35000000120355423,
                                        'amp_end': 0.35000000120355423,
                                        'delay': 270.0,
                                        'duration': 999.9949999999999,
                                        

## Add relaxation around the optimized parameters

In [12]:
select_channels = {
    'gbar_NaTa_t' : ['axonal'],
    'gbar_Kv3_1' : ['somatic', 'axonal', 'apical'],
    'gbar_NaTs2_t' : ['somatic', 'apical'],
    'gbar_K_Tst' : ['somatic', 'axonal', 'apical']
}
tolerance = 0.5

# create bounds for these selected parameters
for param_dict in model_params_bpopt:
    if param_dict['param_name'] in select_channels.keys() and param_dict['sectionlist'] in \
        select_channels[param_dict['param_name']]:
        param_dict['bounds'] = [param_dict['value'] - abs(param_dict['value'])*0.5, 
                                param_dict['value'] + abs(param_dict['value'])*0.5]
        del param_dict['value']

pp.pprint(model_params_bpopt)

bpopt_mech = utility.load_json('config/mechanism.json')

[   {   'dist_type': 'uniform',
        'mech': 'Im',
        'param_name': 'gbar_Im',
        'sectionlist': 'basal',
        'type': 'range',
        'value': 0.00687447546862},
    {   'dist_type': 'uniform',
        'mech': 'Ih',
        'param_name': 'gbar_Ih',
        'sectionlist': 'somatic',
        'type': 'range',
        'value': 1.66485101624e-05},
    {   'dist_type': 'uniform',
        'param_name': 'e_pas',
        'sectionlist': 'all',
        'type': 'section',
        'value': -83.467040608},
    {   'dist_type': 'uniform',
        'param_name': 'cm',
        'sectionlist': 'axonal',
        'type': 'section',
        'value': 5.11878173185},
    {   'dist_type': 'uniform',
        'mech': 'CaDynamics',
        'param_name': 'decay_CaDynamics',
        'sectionlist': 'axonal',
        'type': 'range',
        'value': 914.33998306},
    {   'dist_type': 'uniform',
        'mech': 'Ca_LVA',
        'param_name': 'gbar_Ca_LVA',
        'sectionlist': 'somatic',
        

In [13]:
# Saving the configuration files

config_dir = 'config_dend/'
utility.create_dirpath(config_dir)


for config, config_file in zip([protocols_new, features_new, model_params_bpopt, bpopt_mech], 
                               ['protocols.json', 'features.json', 'parameters.json', 'mechanism.json']):
    
    config_file_path = os.path.join(config_dir, config_file)
    utility.save_json(config_file_path, config)